# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902007


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:28,  3.04s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:12,  2.57s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:50,  1.88s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:36,  1.39s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:15<00:01,  2.83it/s]

Rendering models:  94%|█████████▎| 29/31 [00:15<00:00,  3.33it/s]

Rendering models:  97%|█████████▋| 30/31 [00:15<00:00,  3.29it/s]

Rendering models: 100%|██████████| 31/31 [00:16<00:00,  3.02it/s]

equidad1                              0.112506
loki99                                0.075851
not-logged-in-3391886dcb0cd30b7163    0.001462
WhittleMario                          0.001557
not-logged-in-6f58cd197f9447fe73f1    0.000392
dayanakula                            0.001365
tingard                               0.000212
bobhail                               0.000154
aimee12                               0.001700
LeonardoDB                            0.005864
Khaazel                               0.000241
lsautter                              0.000154
defender1969                          0.004278
Ralfi                                 0.002269
not-logged-in-923f38b394a04ae3a577    0.000375
not-logged-in-314d48142a3a122ba666    0.001361
not-logged-in-dddbb97cfd84354416f5    0.000397
ougb3269                              0.062459
peakscience7                          0.002599
EM4893                                0.000370
not-logged-in-3569dcff4dbd0ee90f50    0.056777
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())